# Fusion

- Deux types de fusions:
  - La fusion verticale non controlée - empilement - (`append`)
  - la fusion horizontale contrôlée - appariement - (`merge`).  


## Append 

- Consiste simplement à ajouter des observations entre plusieurs bases, avec ou non un même jeu de variables. 

![](img6/img1.png)


On va généré les deux bases de données avec la commande `input` (non traité dans cette formation: `help input`)


In [ ]:
clear 
input str6 id v1 v2
  "A" 8 2 
  "B" 1 2 
  "C" 2 4
end

list

save base1, replace

In [ ]:
clear 
input str20 id v1 v2 v3
  "D" 2 5 10
  "E" 12 1 8 
end
list

save base2, replace

La syntaxe de la commande `append` consiste à ajouter une ou plusieurs bases à la base active avec l'argument `using`.  


In [ ]:
append using base1
sort id
list

On peut sélectionner les variables de la base qui sera empilée à la base active avec l'option `keep`.   
Dans l'exemple, si la base active est *base1*, on peut ne pas vouloir ajouter la variable *v3* seulement renseignée pour les observations de *base2*.


In [ ]:
use base1, clear
append using base2, keep(id v1 v2)
list 

Si les informations précédentes étaient ventilées dans trois bases, une par variable *v*, et avec le même niveau d'observation (A,B,C,D,E dans les 3 bases), l'utilisation  de `append` conduirait à une structure empilée non souhaitable avec une réplication des *id*.



Pour obtenir base finale proprement appariée, il convient de faire une fusion horizontale contrôlée par une une clé d'identification.


## Merge

* Fusion contrôlée par une clé d’appariement (variable(s))
* Un merge peut réaliser un empilement (`append`), mais pas l’inverse.
* La commande usine est **`merge`**. Pour contrôler des niveaux d’identification différents on peut y ajouter `1:1`, `1:m`, `m:1` ou `m:m` (le dernier prefixe estrarement ou jamais utiliser). Avec ces prefixes ont ne peut apparier que deux bases.
* Depuis Stata 16 on peut réaliser des merge en transformant les bases en **frame**. Cette solution est très adapté aux appariements de données individuelles avec des données contextuelles. Cette solution sera traitée plus bas.

::: callout-important

Stata demande que les bases soient soit triées (`sort`) sur la clé d'appariement en amont de l'opération. Sinon un message d'erreur sera renvoyé.  
:::

* La base active (ouverte) est appelée **base master**
* La base qui sera appariée à la base ouverte est appelée base **using** ^[Cela peut être plusieurs bases.]


**Syntaxe minimale 1 avec préfixe**:  
```markdown
merge [1:1] [1:m] [m:1] id_var using nom_base
```

* Ici on ne peut pas apparier plus de 2 bases. 
* Si les deux bases ont un niveau d'identification différent, par exemple un contextuel et un individuel, la commande `merge 1:1` sera bloquée. Ces préfixes jouent donc le rôle de sécurité.


**Syntaxe minimale 2 sans  préfixe**:  
```markdown
merge id_var using nom_bases
```

* Ici on peut apparier plus de deux bases. 
* On ne dispose pas de sécurité, si les niveau d'identification sont différents. 

Partons des informations suivantes: 
- *Base1* comprend la variable d'identification *id* (observations A,B,C) et de deux variables numériques *v1* et *v3*
- *Base2* comprend la même variable d'identification *id* (observations B,C,D) et de la variable numérique *v3*

![](img6/img3.png)
Le niveau d'identification est identique dans les deux bases. Il s'agit donc d'un **`merge 1:1`**  [one to one]

On va de nouveau générer les bases avec `input`.  


In [ ]:
clear 
input str1 id v1 v2 
"A" 8 2 
"B" 1 2
"C" 2 4 
end
list

sort id
save base1, replace

Rappel: bien faire le `sort` sur la base using


In [ ]:
clear
input str1 id v3 
"B" 10 
"C" 8
"D" 10 
end
list

sort id
save base2, replace 

Pour l'appariement, de nouveau faire le tri


In [ ]:
merge 1:1 id using base1

* L'output affiche le résultat de l'appariement à l'aide d'un tableau.
* Une variable, **`_merge`**, est générée. Elle permettra de filtrer si nécessaire les observations selon le résultat de l'aapriement. Contrairement à d'autres application, cette opération n'est pas effectuée en amont avec des fonctions où des options spécifiques. Par exemple avec R: `left_join`, `right_join`, `inner_join`.
  `_merge = 1` : observations qui se trouvent seulement dans la base active (master)
  `_merge = 2` : observations qui se trouvent seulement dans la base using (appariée)
  `_merge = 3` : observations communes aux bases master et using.
* Les variables de la base master/active sont positionnées en tête.


In [ ]:
sort id
list

Si on souhaite conserver que les observations communes aux deux bases (`_merge=3`): 


In [ ]:
keep if _merge==3
list

::: callout-warning

## Variable _merge et appariement successif

Pensez à supprimer la variable _merge si plusieurs opérérations d'appariement sont effectués. La commande ne prévoit pas 
d'écraser la variable du merge précédent.
:::




![](img6/img2.png)